# parameters

In [1]:
infile='/Users/nikett/quick/sciencesrl-annotation-afresh/results/sandbox_v0.pkl'
outfile=infile.replace('.pkl','.tsv')

In [2]:
# from pprint import pprint
# import json
# json_response_file="/Users/nikett/quick/sciencesrl-annotation-afresh/results/sample_response.json"
# with open(json_response_file) as json_data:
#       parsed_json = json.load(json_data)
#       ar_dict={}  
#       if "actionRelations" in parsed_json:
#               for ar in parsed_json["actionRelations"]:
#                 ar_dict[str(ar["source"]) +"\t"+ ar["relation"]] = str(ar["target"])
#       for verbid,p in enumerate(parsed_json["actions"]):
#         print("==========================")
#         print("verb"+"\t"+p['verb']["lemma"]+""+str(p['verb']["span"]))
#         for role in ['who','what', 'when','where','whereFrom','whereTo','input','output','enable','prevent']:
#             ar_role = str(verbid)+"\t"+ role
#             if ( ar_role in ar_dict):
#                 print(role+"\t"+ parsed_json["actions"][int(ar_dict[ar_role])]['verb']['lemma']+""+ str(parsed_json["actions"][int(ar_dict[ar_role])]['verb']['span']))
#             elif (role in p['questions'] and p['questions'][role]):
#                 print(role+"\t"+p['questions'][role]["phrase"])
#             else:
#                 print(role+"\t"+"")
#                 #TODO write role values to a TSV file
                

verb	mix[3, 4]
who	
what	Light and water
when	
where	
whereFrom	
whereTo	
input	Light and water
output	sugar
enable	form[5, 6]
prevent	
verb	form[5, 6]
who	
what	sugar
when	
where	
whereFrom	
whereTo	
input	
output	
enable	
prevent	


In [25]:
from pprint import pprint
import json
from mturk_utils.mturk import pickle_this, unpickle_this

def create_result_scisrl(sent_id, assmt):
      parsed_json = json.loads(assmt.answers[0][0].fields[0])
      ar_dict={}
      if "actionRelations" in parsed_json:
              for ar in parsed_json["actionRelations"]:
                ar_dict[str(ar["source"]) +"\t"+ ar["relation"]] = str(ar["target"])
      overall_results = {}              
      for verbid,p in enumerate(parsed_json["actions"]):
        ar_roles = {}
        print("==========================")
        print("sentid: "+str(sent_id) +"\tverbid: "+ str(verbid))
        ar_roles['sentid']=sent_id
        ar_roles['verbid']=verbid
        print("verb"+"\t"+p['verb']["lemma"]+""+str(p['verb']["span"]))
        ar_roles['verb']=p['verb']["lemma"]+""+str(p['verb']["span"])
        for role in ['who','what', 'when','where','whereFrom','whereTo','input','output','enable','prevent']:
            ar_role = str(verbid)+"\t"+ role
            if ( ar_role in ar_dict):
                ar_roles[role]=parsed_json["actions"][int(ar_dict[ar_role])]['verb']['lemma']+""+ str(parsed_json["actions"][int(ar_dict[ar_role])]['verb']['span'])
                print(role+"\t"+ parsed_json["actions"][int(ar_dict[ar_role])]['verb']['lemma']+""+ str(parsed_json["actions"][int(ar_dict[ar_role])]['verb']['span']))
            elif (role in p['questions'] and p['questions'][role]):
                ar_roles[role]=p['questions'][role]["phrase"]
                print(role+"\t"+p['questions'][role]["phrase"])
            else:
                ar_roles[role]=""
                print(role+"\t"+"")
        pprint(ar_roles)
        overall_results[str(sent_id) +"\t"+ str(verbid)] = ar_roles
        return overall_results
                
turk_data = unpickle_this(infile)
assignments = [item for sublist in turk_data.values() for item in sublist]
assignment_results = [create_result_scisrl(sent_id, ar) for sent_id,ar in enumerate(assignments)]
print(assignment_results)

sentid: 0	verbid: 0
verb	transport[1, 2]
who	Plants
what	water
when	
where	from roots to leaves
whereFrom	from roots
whereTo	to leaves
input	
output	
enable	
prevent	
{'enable': '',
 'input': '',
 'output': '',
 'prevent': '',
 'sentid': 0,
 'verb': u'transport[1, 2]',
 'verbid': 0,
 'what': u'water',
 'when': '',
 'where': u'from roots to leaves',
 'whereFrom': u'from roots',
 'whereTo': u'to leaves',
 'who': u'Plants'}
[{'0\t0': {'prevent': '', 'what': u'water', 'enable': '', 'verbid': 0, 'output': '', 'who': u'Plants', 'when': '', 'verb': u'transport[1, 2]', 'whereFrom': u'from roots', 'whereTo': u'to leaves', 'sentid': 0, 'input': '', 'where': u'from roots to leaves'}}]


# load and basic processing

In [ ]:
# import pandas as pd
# from collections import Counter, defaultdict
# import json
# from mturk_utils.mturk import pickle_this, unpickle_this

# def create_result(assmt):
#     result = {}
#     raw_result = json.loads(assmt.answers[0][0].fields[0])
#     result['image_id'] = raw_result['image_url']
#     result['object_words_raw'] = sorted([w.replace('_0_','_').replace('None', 'None_0_0') for w in raw_result['description']], key=lambda x: ''.join(x.split('_')[1:]))
#     obj_words = [word.split('_')[0] for word in result['object_words_raw']]
#     obj_word_location = [word.split('_')[1:] for word in result['object_words_raw']]
#     result['object_words'] = obj_words
#     result['object_locs'] = obj_word_location
#     result['asgmt_id'] = assmt.AssignmentId
#     result['hit_id'] = assmt.HITId
#     result['worker_id'] = assmt.WorkerId
#     return result

# turk_data = unpickle_this(infile)
# print turk_data
# assignments = [item for sublist in turk_data.values() for item in sublist]
# assignment_results = [create_result(ar) for ar in assignments]

# analysis

In [ ]:
def overlap(row):
    for idx in range(row.shape[0]):
        if row[idx] == None:
            row[idx] = set([])
    try:
        return list(row[0].intersection(row[1]).union(row[1].intersection(row[2])).union(row[0].intersection(row[2])))
    except TypeError:
        return {}

def distill_objects(obj_idx_list):
    combined_objects = []
    object_coords = []
    
    for word_position in obj_idx_list:
        split_components = word_position.split('_')
        word, sent_n, word_n = split_components[0], int(split_components[1]), int(split_components[2])
        object_coords.append((word, sent_n, word_n))
    object_coords = sorted(object_coords, key= lambda x: (x[1], x[2]))
    combined_objects.append(object_coords[0])
    
    for idx in range(1, len(object_coords)):
        this_word, this_sent, this_wn = object_coords[idx]
        last_word, last_sent, last_wn = combined_objects[-1]
        if this_sent == last_sent and last_wn + 1 == this_wn:            
            combined_objects[-1] = (' '.join([last_word, this_word]), this_sent, this_wn)
        else:
            combined_objects.append((this_word, this_sent, this_wn))
    return [w[1:] for w in combined_objects], [w[0] for w in combined_objects]

# Generates spans from words with word ids in text.
# e.g., words_input: [u'change_10', u'stay_23', u'the_24', u'same_25', u'may_9']
# returns: [u'may_9 change_10', u'stay_23 the_24 same_25']
def spans_from(words_input):
    # step1: sort words by position
    # u'change_10', u'stay_23', u'may_9' => u'may_9', u'change_10', u'stay_23'
    sorted_words = words_input
    sorted_words.sort(key=lambda x: int(x.split('_')[1]))
    # e.g., u'may_9', u'change_10', u'stay_23' (being sorted now)
    spans_from_words = ""
    for i in range(0,len(sorted_words)):
        prev_word_position = i-1 if(i>0) else 0
        # extract word index (e.g., 23 from stay_23) to compare adjacent (sorted) words
        curr_index = (int(sorted_words[i].split('_')[1]))
        prev_index = (int(sorted_words[prev_word_position].split('_')[1]))
        # insert comma at span boundaries.
        spans_from_words += ("," if(curr_index - prev_index >1 and i>0) else " ") + (sorted_words[i])
    return spans_from_words.strip().split(',')

st4_df = pd.DataFrame(assignment_results)
spans = [spans_from(st4_df['object_words_raw'][i]) for i in range(0, st4_df['object_words_raw'].count())]
# insert spans as the last column.
st4_df['spans'] = spans
st4_df.to_csv(outfile, sep='\t')
print "Result in " + outfile

In [ ]:
# Distill good workers.
# Find turkers who had proposed "None" for most of the hits where others didn't.
# st4_df[st4_df['worker_id']=='A3EAOSOUQZFVFS']
st4_df[st4_df['worker_id']=='A3EAOSOUQZFVFS']
# sorted(st4_df[st4_df['object_words'].apply(lambda x: x[0]) == 'None']['worker_id'])

df = st4_df[['image_id', 'spans']]
df.to_csv('/tmp/t.tsv', sep='\t')
# my_dict={
# 'dummy':{'w':0} # don't know how else to initialize.
# }
# # my_dict.update({'item4': {'a':2, 'b':3}, 'item5':  {'a':20, 'b':30}})
# for s in df[df['image_id']=='7']['spans']:
#     my_dict['7'][s]= 1
# my_dict


# st4_df[['image_id', 'spans']].groupby('image_id').agg(lambda x: ';;'.join(set(x)))
# TODO: per image id, aggregate all spans that have support of two or more.
# TODO: check against manual gold.
#.groupby('image_id').agg(lambda x: ';;'.join(set(x)))

# Code in scala and manual filtering is here: /Users/nikett/Documents/work/code/kb-organization/extraction/aristoKB/src/main/scala/eventsAndProcesses/to_be_fixed/predicateid/PredicateIDTurked.scala



sentence_annotations = defaultdict(lambda: defaultdict(int))

for i, id in enumerate(sendid):
    sentence_annnotations[id][annotation] += 1

# filtering step
# using dict comprehension here doesn't look good and gets a little complicated

for key, value in sentence_annotations.keys():
     sentence_annotation[key] = dict((key, value) for key, value in sent_annotation[key].items() if value > 1)